In [39]:
import json
with open('ebird_config.json') as f:
    data = json.load(f)
    recent_species_observations = data['recent_species_observations']
    ebird_token = data['ebird_token']
    gmap_api_key = data['gmap_api_key']
# recent_species_observations

In [40]:
import requests
headers = {}
headers['X-eBirdApiToken'] = ebird_token
response = requests.get(recent_species_observations, headers=headers)
res_json = response.json()

In [41]:
# res_json

In [42]:
import datetime as dt
import collections as col
# some observation date has no HH:MM part, just dates
DATE_FORMAT = '%Y-%m-%d'
Sighting = col.namedtuple('Sighting', ['lat', 'lng', 'date'])
# 0-th is today, 1-st is yesterday, etc
sightings = list()
for sighting in res_json:
    sightings.append(Sighting(lat=sighting['lat'], lng=sighting['lng'],
                              date=dt.datetime.strptime(sighting['obsDt'][:10], DATE_FORMAT)))
today = dt.date.today()
sightings_last_seven_days = list() # today is included
for i in range(0, 7):
    sightings_last_seven_days.append(
        list(filter(lambda s: s.date.date() == today - dt.timedelta(days=i), sightings)))
#sightings_last_seven_days

In [43]:
layers = list()
yx_layers = list()
# colors of the rainbow, the more red the color, the closer to today
colors = list(['#ff0000', '#ffa500', '#ffff00', '#008000', '#0000ff', '#4b0082', '#ee82ee'])
for day_sightings in sightings_last_seven_days:
    layers.append(list(map(lambda s: (s.lat, s.lng), day_sightings)))
    yx_layers.append(list(map(lambda s: (s.lng, s.lat), day_sightings)))

In [44]:
import gmaps
gmaps.configure(api_key=gmap_api_key)
fig = gmaps.figure()
def add_layer(lat_lng, color, scale):
    layer = gmaps.symbol_layer(lat_lng, fill_color=color, stroke_color=color, scale=scale)
    fig.add_layer(layer)
# not counting today, used to further limit the 7 days to a smaller number of days, if desired
#days_back = int(input('Number of days to show, 0 for today only '))
days_back = 3
days_back = max(0, days_back)
# add the layers backwards so today is on top, but it does not seem to work,
# looks like there is no support for setting zIndex on a symbol (or symbols of a layer)
for i in range(min(days_back, 6), -1, -1):
    add_layer(layers[i], colors[i], 2)
    # this line changes the size of the dots according to how recent they are
    #add_layer(layers[i], colors[i], (4 - i) if (days_back <= 3) else int((8-i)/2))
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
%matplotlib inline

In [49]:
%%capture
# turn off output for this cell so that we don't get a standalone map, we just need to see the animation at the end
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gp
from matplotlib import animation, rc
#from IPython.display import HTML

fig, ax = plt.subplots(figsize=(9, 7))
ax.set_xlim((-100, -70))
ax.set_ylim((22, 42))

plural = 's' if days_back > 1 else ''
plt.title(f"Palm Warbler ebird sightings in the last {days_back} day{plural} and today (red)")

states = gp.read_file('../data/geo/us-census/cb_2018_us_state_5m.shp')
states.boundary.plot(ax=ax, color='#bbbbbb')

# sp for scatter plot
sp = ax.scatter([], [])

In [50]:
def update_frame(day, layers, colors, sp):
    sp.set_offsets(np.array(yx_layers[day]))
    sp.set_color(colors[day])
    return sp,

ani = animation.FuncAnimation(fig, update_frame, range(days_back, -1, -1), fargs=(layers, colors, sp),
                              interval=500, blit=True)

#HTML(ani.to_html5_video())
rc('animation', html='html5')
ani